In [1]:
import numpy as np
import pandas as pd
from os import path
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import glob
from tqdm.notebook import tqdm
from bs4 import BeautifulSoup
import requests
import random
import pickle
import re
import spacy
nlp = spacy.load('en_core_web_trf')
import collections
from collections import Counter
from tqdm.notebook import tqdm as tqdm_notebook
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from transformers import DistilBertTokenizer, DistilBertModel
from matplotlib.figure import Figure
from matplotlib import cm
import matplotlib.colors as colors
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

import sys
sys.path.insert(0, '../src/models/')
sys.path.insert(0, '../src/features/')

import predict_model
from predict_model import loadBERT
from predict_model import SpanPredictor as classify
from build_features import text_cleaner
%matplotlib inline

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.bias']
- T

In [2]:
model = loadBERT("../models/", 'saved_weights_inf_FIXED_boot_beta80.pt')
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

CPU Success


In [23]:
def compound_reconstructor(token, doc):
    if token.i == 0:
        trait = token
    elif doc[token.i - 3].dep_ == 'compound':
        trait = doc[token.i - 3: token.i + 1]
    elif doc[token.i - 3].text.lower() in compounds or doc[token.i - 3].lemma_.lower() in compounds:
        trait = doc[token.i - 3: token.i + 1]
    elif doc[token.i - 2].dep_ == 'compound':
        trait = doc[token.i - 2: token.i + 1]
    elif doc[token.i - 2].text.lower() in compounds or doc[token.i - 3].lemma_.lower() in compounds:
        trait = doc[token.i - 2: token.i + 1]
    elif doc[token.i - 1].dep_ == 'compound':
        trait = doc[token.i - 1: token.i + 1]
    elif doc[token.i - 1].text.lower() in compounds or doc[token.i - 3].lemma_.lower() in compounds:
        trait = doc[token.i - 1: token.i + 1]
    else:
        trait = token   
    return trait.lemma_

def check_existance(t, doc, glossary):
    if t.lemma_ in glossary:
        return t
    elif t.text in glossary:
        return t
    else:
        return None

def extract_advmod(t, doc):
    """HELPER"""
    for child in t.children:
        if child.dep_ == 'advmod':
            return child
        
def extract_nummod(t, doc):
    """HELPER"""
    for child in t.children:
        if child.dep_ == 'nummod':
            return child

def extract_conjunction(t, doc):
    """HELPER"""
    if t.dep_ == 'conj' and t.pos_ == 'ADJ':
        return t 

def extract_amod(t, doc):
    """HELPER"""
    for child in t.children:
        if child.dep_ == 'amod':
            return child
        
def extract_measurements(t, doc):
    """HELPER"""
    obj = None
    measurements = ['wide', 'long']
    if t.text in measurements or t.lemma_ in measurements:
        obj = doc[t.left_edge.i : t.right_edge.i + 1]
    return obj

def extract_prepositions(t, doc):
    """HELPER"""
    for child in t.children:
        if child.dep_ == 'prep':
            return doc[child.left_edge.i : child.right_edge.i + 1]

def define_position(x, y, doc):
    """HELPER"""
    if len(x.text.split()) > 1:
        return f'{y.text} {x.text}'
    else:
        try:
            if x.i > y.i:
                return doc[y.i : x.i + 1]
            else:
                return doc[x.i : y.i + 1]
        except:
            return f'{y.text} {x.text}'

def extract_noun_verbs(t, doc):
    relations = []
    objects = []
    if t.dep_ not in ['ROOT', 'nsubj', 'nsubjpass', 'csubj', 'csubjpass']:
        return '', ''
    parent = next((parent for parent in t.ancestors), None)
    if parent and parent.pos_ == 'VERB':
        prep = extract_verb_preps(parent, doc)
        dobj = extract_verb_dobj(parent, doc)
        oprd = extract_verb_orpd(parent, doc)
        agnt = extract_verb_agnt(parent, doc)
        nmod = extract_verb_nmod(parent, doc)
        if prep:
            obj = extract_verb_pobj(prep, doc)
            if obj:
                relations.append(f'{parent.text} {prep}')
                objects.append(obj.lemma_)
        if dobj:
            pass
            #print(dobj)
        if oprd:
            pass
            #print(oprd)
        if agnt:
            pass
           # print(agnt)
        if nmod:
            pass
           # print(nmod)
        grandparent = next((grandparent for grandparent in parent.ancestors), None)
        if not grandparent:
            relations.append('is')
            numbers = extract_nounandverb_nummods(parent, doc)
            if numbers:
                objects.append(f'{numbers.text} {parent.text}')
            else:
                objects.append(parent.text)
    if not parent:
        for child in t.children:
            if child.pos_ == 'VERB' and child.dep_ != 'amod':
                                
                prep = extract_verb_preps(child, doc)
                dobj = extract_verb_dobj(child, doc)
                oprd = extract_verb_orpd(child, doc)
                agnt = extract_verb_agnt(child, doc)
                nmod = extract_verb_nmod(child, doc)
                if prep:
                    noun = extract_verb_pobj(prep, doc)
                    if noun:
                        relations.append(f'{child.text} {prep}')
                    
                        objects.append(doc[noun.left_edge.i : noun.right_edge.i + 1].text)
                if dobj:
                    relations.append(child.text)
                    objects.append(doc[dobj.left_edge.i : dobj.right_edge.i + 1].text)     
                if oprd:
                    oprd_prep = extract_verb_preps(oprd, doc)
                    if oprd_prep:
                        relations.append(f'{child.text} {oprd.text}')
                        objects.append(doc[oprd_prep.left_edge.i : oprd_prep.right_edge.i + 1].text)  
                if agnt:
                    continue
                    #print(agnt)
                if nmod:
                    relations.append('is')
                    objects.append(f'{nmod.text} {child}') 

    rel = []
    obj = []
    for relation, object_ in zip(relations, objects):
        rel.append(relation)
        obj.append(object_.split(',')[0])
            
    return rel, obj

def extract_verb_nmod(t, doc):
    """HELPER"""
    for child in t.children:
        if child.dep_ == 'nummod':
            return doc[child.left_edge.i : child.right_edge.i + 1] 
        
def extract_verb_preps(t, doc):
    """HELPER"""
    for child in t.children:
        if child.dep_ == 'prep':
            return child    

def extract_verb_pobj(t, doc):
    """HELPER"""
    for child in t.children:
        if child.dep_ == 'pobj' or child.dep_ == 'pcomp' or child.dep_ == 'prep':
            return child
        
def extract_verb_dobj(t, doc):
    """HELPER"""
    for child in t.children:
        if child.dep_ == 'dobj':
            return child
        
def extract_verb_orpd(t, doc):
    """HELPER"""
    for child in t.children:
        if child.dep_ == 'oprd':
            return child    
        
def extract_verb_agnt(t, doc):
    """HELPER"""
    for child in t.children:
        if child.dep_ == 'agent':
            return child 
        
def extract_nounandverb_nummods(t, doc):
    obj = None
    for child in t.children:
        if child.dep_ == 'nummod':
            obj = doc[child.left_edge.i : child.right_edge.i + 1]
            return obj   

def extract_dnummod(t, doc):
    obj = extract_nounandverb_nummods(t, doc)
    if obj:
        return obj.text
    else:
        return None

def extract_noun_adjectives(t, doc):
    adjs = []
    adjectives = []
    for child in t.children:
        if child.dep_ == 'appos':
            continue
        if child.pos_ =='ADJ' or child.tag_ == 'VBN' and child.dep_ in ['conj', 'amod']:
            
            
            advmod = extract_advmod(child, doc)
            measurement = extract_measurements(child, doc)
            prep = extract_prepositions(child, doc)
            nummod = extract_nummod(child, doc)
            amod = extract_amod(child, doc)
           
            
            if child.lemma_.lower() in compounds:
                continue
            if child.text.lower() in compounds:
                continue
            elif amod:
                obj = define_position(amod, child, doc)
                adjs.append(obj)
            elif advmod:
                #obj = doc[advmod.i : child.i + 1]
                obj = define_position(advmod, child, doc)
                adjs.append(obj)
            elif measurement:
                obj = measurement
                adjs.append(obj)
            elif prep:
                obj = define_position(prep, child, doc)
                adjs.append(obj)
            elif nummod:
                obj = define_position(nummod, child, doc)
                adjs.append(obj)                
            else:
                obj = child
                adjs.append(obj)
            for grandchild in child.subtree:
                conj = extract_conjunction(grandchild, doc)
                if conj:
                    advmod = extract_advmod(conj, doc)
                    prep = extract_prepositions(conj, doc)
                    nummod = extract_nummod(conj, doc)
                    if advmod:
                        obj = define_position(advmod, conj, doc)
                        adjs.append(obj)
                    elif prep:
                        obj = define_position(prep, conj, doc)
                        adjs.append(obj)
                    elif nummod:
                        obj = define_position(nummod, conj, doc)
                        adjs.append(obj)
                    else:
                        obj = conj
                        adjs.append(obj)            
    for adj in adjs:
        try:
            if adj.pos_ == 'VERB':
                adj_text = adj.text.lower()
            elif adj.root.pos_ == 'VERB':
                adj_text = adj.text.lower()
            else:
                adj_text = adj.lemma_.lower()
        except:
                if type(adj) == str:
                    adj_text = adj.lower()
                else:
                    adj_text = adj.text.lower()
        for adj_split in adj_text.split(','):
            adjectives.append(adj_split.strip())
    return adjectives

def extract_noun_appos(t, doc):
    appos = []
    for child in t.children:
        if child.dep_ == 'appos':
            obj = doc[child.left_edge.i : child.right_edge.i + 1].text.lower()
            for obj_split in obj.split(','):
                appos.append(obj_split.strip())
    return appos

def check_species(t, species, doc):
    if t.text in species.split():
        return True
    else:
        return False
    
def extract_auxillary(t, doc):
     parent = next((parent for parent in t.ancestors if parent.pos_ == 'AUX'), None)

In [24]:
# URL
URL = 'https://en.wikipedia.org/wiki/Glossary_of_bird_terms'
# Get the page
page = requests.get(URL, timeout=5)
soup = BeautifulSoup(page.content, "html.parser", from_encoding="iso-8859-1")   
# Find embedded glossary
glossaries = soup.find_all('dt', {'class': 'glossary'})
parts = [part.text.lower().strip() for part in glossaries]
# Get additional anchors ("also know as...")
glossaries_other = soup.find_all('span', {'class': 'anchor'})
parts_other = [part['id'].lower().strip() for part in glossaries_other]
# Append and drop duplicates
parts = list(set((parts + parts_other)))
# Replace underscore with space
glossary = [part.replace('_', ' ') for part in parts]


# A few helpers
additions = [
    'legs',
    'beak',
    'head',
    'wingspan',
    'eye',
    'forecrown'
]

glossary += additions

# Compound helpers
compounds = [
    'upper', 'lower', 'greater', 'dorsal', 'alternate', 'lesser',
    'central', 'outermost', 'outer', 'inner', 'uppermost', 'median',  
    'upperwing', 'underwing', 'tail', 'rump',
    'secondary', 'primary', 'definitive', 'basic', 'tertial', 'preformative', 'prebasic', 'tertials',
    'posterior', 'fresh', 
    'sexual', 'juvenile', 'male', 'female', 'adult', 'juvenal',
    'breeding', 'predefinitive', 'non-breeding',
    'raised', 'closed', 'open', 
    'entire', 'partial',
    

]

In [25]:
URL = 'https://en.wikipedia.org/wiki/List_of_birds_of_the_Netherlands'
# Get the page
page = requests.get(URL, timeout=5)
soup = BeautifulSoup(page.content, "html.parser", from_encoding="iso-8859-1") 

species_NL = []
for candidate in soup.find_all('li'):
    species = candidate.text.split(',')[0]
    species_NL.append(species.lower())

In [26]:
# Read all html files
species_folder = glob.glob('../data/raw/BOW/*')
species_ALL = [link[16:].lower() for link in species_folder]

species_NL = list(set(species_ALL) & set(species_NL))

In [27]:
data = pickle.load(open('../data/processed/Dutch_birds_descriptions.pkl', 'rb'))

In [28]:
descriptions = collections.defaultdict(list)
descriptions_cleaned = collections.defaultdict(list)

for dutch_bird in tqdm_notebook(species_NL[0:20]):
    idx = species_ALL.index(dutch_bird)
    files = glob.glob(species_folder[idx] + '/*')
    for file in files:
        with open(file) as f:
            soup = BeautifulSoup(f, 'html.parser')
            dirty_text = soup.get_text(" ", strip=False).replace('\n', '.')
            # Clean and break into sents
            sentences = text_cleaner(dirty_text)

            for idx, sentence in enumerate(sentences):
                if classify(sentence, model=model):
                    doc = nlp(sentence)
                    # Init to check empty values
                    descriptions[dutch_bird, idx] = []
                    triples = []
                    # Loop over tokens
                    for t in doc:
                        if t.dep_ == 'compound':
                            continue
                        ### SUBJECTS ###    
                        if t.pos_ == 'NOUN' or t.pos_ == 'PROPN':
                            # Check existance of parts
                            part = check_existance(t, doc, glossary=glossary)
                            if part:
                                # Reconstruct Compounds & Append
                                trait = compound_reconstructor(t, doc)
                                triples.append(('bird', 'has part', trait))
                                # NOUN ADJECTIVES
                                adjectives = extract_noun_adjectives(t, doc)
                                for adjective in adjectives:
                                    triples.append((trait, 'is', adjective))
                                # NOUN VERBS
                                verbs_rel, verbs_obj = extract_noun_verbs(t, doc)
                                for rel, obj in zip(verbs_rel, verbs_obj):
                                    triples.append((trait, rel, obj))
                                # NOUN APPOSITIONAL MODIFIER
                                adjectives = extract_noun_appos(t, doc)
                                for adjective in adjectives:
                                    triples.append((trait, 'is', adjective))
                                # NOUN NUMMODS
                                nummod = extract_dnummod(t, doc)
                                triples.append((trait, 'is', nummod))

                        #if check_species(t, species, doc):

                    # APPEND
                    descriptions[dutch_bird, idx] = [triple for triple in triples if all(triple)]     
                            

  

  0%|          | 0/20 [00:00<?, ?it/s]

In [29]:
with open('../data/processed/dutch_birdv2_unclean.pkl', 'wb') as f:
    pickle.dump(descriptions, f)   

In [30]:
descriptions_cleaned = collections.defaultdict(list)

for (bird, idx) in descriptions.keys():
    for (sub, rel, obj) in descriptions[(bird, idx)]:
        if rel != 'has part':
            descriptions_cleaned[bird].append(f'{sub} {obj}')

In [31]:
#descriptions_cleaned['sooty tern']

In [32]:
with open('../data/processed/dutch_birdv2_clean.pkl', 'wb') as f:
    pickle.dump(descriptions_cleaned, f)   